# Implementing Page Rank on Phenotype-GARD disease graph

In [5]:
from py2neo import Graph
from py2neo import Node

In [6]:
import pandas as pd
from matplotlib import pyplot as plt

In [7]:
from py2neo import Database
db = Database("bolt://disease.ncats.io:80")
graph = db.default_graph

In [4]:
#get networkx subset
#use networkx page rank algo

In [8]:
query = """match (n:S_GARD)--(d:DATA) where d.id = 5 with n,d 
        match p=(n)-[R_rel{name:'has_phenotype'}]-
        (:S_HP)--(:DATA) return p,d"""

In [9]:
cursor = graph.run(query)

#print the records from the Database.cursor object
for record in cursor:
    print(record)

<Record p=(_7)-[:R_rel {`HPO-ID`: 'HP:0001252', `HPO-term-name`: 'Muscular hypotonia', created: 1591077694718, diseaseId: 'ORPHA:14', `gene-id(entrez)`: '4547', `gene-symbol`: 'MTTP', name: 'has_phenotype', source: 'a8cf83e5f', value: 'HP:0001252'}]->(_66781)<-[:PAYLOAD {id: 'http://purl.obolibrary.org/obo/HP_0001252', source: 'd7a8dc9fd'}]-(_66782) d=(_8:DATA {Cause: 'Abetalipoproteinemia is caused by changes (<a href="http://www.genome.gov/glossary/index.cfm?id=134" target="_blank">mutations</a>) in the <a href="http://ghr.nlm.nih.gov/gene/MTTP" target="_blank"><em>MTTP</em></a> gene. The <em>MTTP</em> gene gives the body instructions to make a <a href="http://ghr.nlm.nih.gov/handbook/howgeneswork/protein" target="_blank">protein</a> needed for creating beta-lipoproteins. These lipoproteins are necessary for the body to absorb fats, cholesterol, and fat-soluble vitamins (vitamins A,&nbsp;D, E and K), and for transporting these substances&nbsp;in the blood. Mutations in the <em>MTTP</

<Record p=(_7)-[:R_rel {`HPO-ID`: 'HP:0002630', `HPO-term-name`: 'Fat malabsorption', created: 1591072667542, diseaseId: 'OMIM:200100', `gene-id(entrez)`: '4547', `gene-symbol`: 'MTTP', name: 'has_phenotype', source: 'a8cf83e5f', value: 'HP:0002630'}]->(_69462)<-[:PAYLOAD {id: 'http://purl.obolibrary.org/obo/HP_0002630', source: 'd7a8dc9fd'}]-(_69463) d=(_8:DATA {Cause: 'Abetalipoproteinemia is caused by changes (<a href="http://www.genome.gov/glossary/index.cfm?id=134" target="_blank">mutations</a>) in the <a href="http://ghr.nlm.nih.gov/gene/MTTP" target="_blank"><em>MTTP</em></a> gene. The <em>MTTP</em> gene gives the body instructions to make a <a href="http://ghr.nlm.nih.gov/handbook/howgeneswork/protein" target="_blank">protein</a> needed for creating beta-lipoproteins. These lipoproteins are necessary for the body to absorb fats, cholesterol, and fat-soluble vitamins (vitamins A,&nbsp;D, E and K), and for transporting these substances&nbsp;in the blood. Mutations in the <em>MTTP

In [13]:
from neo4j import GraphDatabase
from neo4j.types.graph import Node, Relationship
import pandas as pd
import networkx as nx

In [14]:
def graph_from_cypher(data):
    """Constructs a networkx graph from the results of a neo4j cypher query.
    Example of use:
    >>> result = session.run(query)
    >>> G = graph_from_cypher(result.data())

    Nodes have fields 'labels' (frozenset) and 'properties' (dicts). Node IDs correspond to the neo4j graph.
    Edges have fields 'type_' (string) denoting the type of relation, and 'properties' (dict)."""

    G = nx.MultiDiGraph()
    def add_node(node):
        # Adds node id it hasn't already been added
        u = node.id
        if G.has_node(u):
            return
        G.add_node(u, labels=node._labels, properties=dict(node))

    def add_edge(relation):
        # Adds edge if it hasn't already been added.
        # Make sure the nodes at both ends are created
        for node in (relation.start_node, relation.end_node):
            add_node(node)
        # Check if edge already exists
        u = relation.start_node.id
        v = relation.end_node.id
        eid = relation.id
        if G.has_edge(u, v, key=eid):
            return
        # If not, create it
        G.add_edge(u, v, key=eid, type_=relation.type, properties=dict(relation))

    for d in data:
        for entry in d.values():
            # Parse node
            if isinstance(entry, Node):
                add_node(entry)
                print("node")

            # Parse link
            elif isinstance(entry, Relationship):
                add_edge(entry)
                print("edge")
            else:
                raise TypeError("Unrecognized object")
    return G

In [43]:
driver = GraphDatabase.driver("bolt://disease.ncats.io:80", auth=("neo4j", ""))

In [27]:
%%time
query = """match r=(d:DATA)--(n:S_GARD)-[:R_rel{name:"has_phenotype"}]-(p:S_HP)--(b:DATA) return *"""

with driver.session() as session:
    result = session.run(query)

CPU times: user 1min 43s, sys: 4.2 s, total: 1min 47s
Wall time: 2h 15min 50s


In [ ]:
%%time
query = """match (n:S_GARD)--(d:DATA) with n,d match r=(n)-[:R_rel{name:"has_phenotype"}]-(p:S_HP)--(b:DATA) 
        return r,d,b"""

with driver.session() as session:
    result = session.run(query)

In [39]:
graph = graph_from_cypher(result.data())

TypeError: Unrecognized object

In [32]:
graph2 = graph_from_cypher(result.data())

In [33]:
graph2.number_of_nodes()

0

In [41]:
print(nx.info(graph))

Name: 
Type: MultiDiGraph
Number of nodes: 0
Number of edges: 0



In [40]:
result.da

In [ ]:
#